In [3]:
# email2inventory
# read evergreen email
import os
import re

def evergreen_read(filename):
    date = ""
    product_flag = False
    position = 0
    items = {}
    f = open(filename)
    lines = f.readlines()
    while not product_flag:
        line = lines.pop(0).strip()
        m = re.match(r"Date: (\w* \d+, \d{4})", line)
        if m:
            date = m.group(1)
        if line == "Product":
            product_flag = True
    line = lines.pop(0).strip()
    assert line == "Quantity"
    line = lines.pop(0).strip()
    assert line == "Price"
    while product_flag:
        m = re.match(r"(.*) - 1 Week", lines.pop(0).strip())
        try:
            itemname = m.group(1)
        except:
            break
        quantity = lines.pop(0).strip()
        price = lines.pop(0).strip()
        items[itemname] = [quantity,price]
    f.close()
    return(items, date)
    
print(evergreen_read(os.path.join(os.getenv("HOME"), "scripts/email2inventory/data/evergreen-20200930.txt")))

({"Red's Best Hake": ['2', '$32.00']}, 'September 30, 2020')


This uses OAUTH2 to get the two folders from the Sortly API and then open an item and get the image.

Boris figured out how to do it using `urllib.request`, but I think I should be able to do it using `requests`.  Also, I think we're doing some sort of brute force OAUTH and that there is something more complicated but proper - maybe using requests. Something to try to figure out.

I moved the client key to sortly_key.sh as a file that prints the key. I ignored it in GitHub and am not syncing it.

In [11]:
# trying to access Sortly API
import urllib.request
import json
import urllib
import requests
import io
import subprocess
from PIL import Image

client_secret = subprocess.check_output("./sortly_key.sh").strip()
print(client_secret)

headers = {
  'Accept': 'application/json',
  'Authorization': client_secret
}
# trying to use request instead of urllib but can't figure it out
# request = requests.get('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers)
# print(request)
# response_py = request.json()
request = urllib.request.Request('https://api.sortly.co/api/v1/items?per_page=&page=&folder_id=&include=', headers=headers)
response_body = urllib.request.urlopen(request).read()
response_py = json.loads(response_body)
print(response_py)
next_url = response_py['meta']['next_page_url']
request = urllib.request.Request(next_url, headers=headers)
response_body = urllib.request.urlopen(request).read()
response_py = json.loads(response_body)
print(response_py)
next_url = response_py['meta']['next_page_url']
request = urllib.request.Request('https://api.sortly.co/api/v1/items/17481596?include=custom_attributes%2Cphotos', headers=headers)
response_body = urllib.request.urlopen(request).read()
response_py = json.loads(response_body)
print(response_py)
photo_url = response_py['data']['photos'][0]['url']
response = requests.get(photo_url)
img = Image.open(io.BytesIO(response.content))
img.show()

CalledProcessError: Command './sortly_key.sh' returned non-zero exit status 1.